# Simple VGG16

Check the devices available. Note you need to have the right version (as in CPU vs GPU version) of TensorFlow installed to harnes the GPU. 

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'
tf.Session(config = config)

C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3871148138084432590
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3404103680
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1993367868355839448
physical_device_desc: "device: 0, name: GeForce GTX 970, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


This can take a bit of time if it's the first time you're running it. It will download a Keras model equivalent to VGG-16.

In [2]:
import keras
import numpy as np
from keras.applications import vgg16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model

#Load the VGG model
vgg_model = vgg16.VGG16(weights='imagenet')


Using TensorFlow backend.


# Train Model
Changing the last layer from 1000 prediction to 37 predictions
my_model is the vgg model if you want to use it later

In [3]:
from keras.optimizers import RMSprop

#Add a layer where input is the output of the  second last layer 
x = Dense(37, activation='softmax', name='predictions')(vgg_model.layers[-2].output)

#Then create the corresponding model 

optimizer = RMSprop(lr=1e-6)
my_model = Model(input=vgg_model.input, output=x)
my_model.compile(loss='mean_squared_error', optimizer=optimizer)
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  if __name__ == '__main__':


# Paths
Remember to check for the path of the directory

In [4]:
import glob

def load_img_paths(target):
    '''
    Retrieve the full path of all images in the original training dataset
    '''
    return glob.glob(target + '/*.jpg')

data_dir = r'../data'
original_data_dir = data_dir + '/images_training_rev1'
training_dir      = data_dir + '/training'
validation_dir    = data_dir + '/validation'
test_dir          = data_dir + '/test'

# Load image file names
train_paths = load_img_paths(training_dir)
valid_paths = load_img_paths(validation_dir)
test_paths = load_img_paths(test_dir)
train_paths[0:5]

['../data/training\\100023.jpg',
 '../data/training\\100090.jpg',
 '../data/training\\100237.jpg',
 '../data/training\\100367.jpg',
 '../data/training\\100380.jpg']

In [5]:
from keras.preprocessing.image import load_img

image = load_img(train_paths[0], target_size=(224, 224))

from keras.preprocessing.image import img_to_array
# convert the image pixels to a numpy array
image = img_to_array(image)

The network expects one or more images as input; that means the input array will need to be 4-dimensional: samples, rows, columns, and channels.

We only have one sample (one image). We can reshape the array by calling reshape() and adding the extra dimension.

In [6]:
from keras.applications.vgg16 import preprocess_input
import pandas as pd
import numpy as np

# # reshape data for the model
# image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# # prepare the image for the VGG model
# image = preprocess_input(image)

# # predict the probability across all output classes
# output = my_model.predict(image)

df = pd.read_csv(data_dir + '/training_solutions_rev1.csv')
df.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


# Extracting
Getting the training data number from the path

In [7]:
from os import path # removes confusion with / and \ (windows)
file_id = path.split(train_paths[0])[-1] \
              .split('.')[0] # remove file extension
fd = int(file_id)

row = df.loc[df['GalaxyID'] == fd].loc[:,df.columns != 'GalaxyID']
row

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
1,0.327001,0.663777,0.009222,0.031178,0.632599,0.46737,0.165229,0.591328,0.041271,0.0,...,0.018764,0.0,0.131378,0.45995,0.0,0.591328,0.0,0.0,0.0,0.0


## Training

In [8]:
def fetch_images(paths):
    for fname in paths:
        image = load_img(fname, target_size=(224, 224))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        train = preprocess_input(image)
        file_id = path.split(fname)[-1] \
                      .split('.')[0]
        fd = int(file_id)
        test = df.loc[df['GalaxyID'] == fd].loc[:,df.columns != 'GalaxyID']
        yield (train, [test])
        

In [9]:
vgg = my_model # cache my model so if vgg.fit has some result we don't override it.

In [ ]:
vgg = vgg.fit_generator(fetch_images(train_paths),
    steps_per_epoch=1000, 
    nb_epoch=50,
    validation_data=fetch_images(valid_paths),
    nb_val_samples=500,
    verbose=2
    #callbacks=[history,checkpointer,early_stopping],
)

C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=1000, validation_data=<generator..., verbose=2, epochs=50, validation_steps=500)`
  


Epoch 1/50
